à installer avant d'executer le programme :
- pip install folium
- pip install --upgrade gtfs-realtime-bindings
- pip install requests
- io?????
- pip install zipp
- pip install urllib3

Explication dans l'ensemble du programme:

   Question auquel le progamme veut repondre à quel point le prochain bus que je veut prendre est loin de mon arrêt
    
   Pour ce faire, on va demander à l'utilisateur de rentrer la ligne qui l'intéresse et l'arrêt auquel il est.
   
   Ensuite on va créer un dataframe à partir des donnéees en temps réel. Ce dataframe aura la liste de tous les bus qui roulent en direct sur la ligne qui nous intéresse. 
   
   Aussi on va créer un datafame qui contiendra les information dites statiques(les coordonnées de l'arret, sa séquence dans le trajet du bus, pour la ligne entrée par l'utilisateur.
   
   Ensuite on va fusionner les dataframe pour avoir toutes les données dans un seule dataframe. Cela devrait nous permettre de nous faciliter la vie pour les calculs à faire.
   
   On veut savoir/calculer deux choses : la distance en arrêts restants et en kilomètre. POur ce faire on premièrement soustraire la stop_sequence de l'arret à celle du bus et deuxièmement utiliser une fonction trouvée sur internet pour calculer une distance en kilomètre.
    
   On veut ensuite afficher dans folium la position du bus le plus proche. On va créer 2 markers (un sera l'arrêt et l'autre le bus). On va juste pendre les coordonnées du bus le plus haut dans le tableau qui sera le plus proche puisque on aura précédement trié par la distance en arrêts restants.
   
   Et voilà ! on a sur une carte la position de notre bus le plus proche de notre arrêt !!!!!
    

In [14]:
import requests #pour nous permettre de faire des requêtes sur internet
from google.transit import gtfs_realtime_pb2 #pour nous permettre d'acceder à la donnée en temps réel
from google.protobuf.json_format import MessageToJson #pour convertir la donnée en temps réel en sort de json
import pandas #pour pouvoir faire des jolis DataFrame
from io import BytesIO #pour lire le contenu d'un fichier compressé
from zipfile import ZipFile #pour pouvoir décompresser un fichier zip
import urllib.request #pour pouvoir faire des requete en ligne
import math #pour effectuer des calculs nécessitant des fonctions plus poussées
import folium #pour afficher la map

In [7]:
#on crée la fonction qui calculera la distance entre deux pairs de coordonées en dégrés décimaux
def distance(lat1,lon1,lat2,lon2): #on a besoins de 4 variables pour effectuer les calculs
    #https://gist.github.com/rochacbruno/2883505
    radius = 6371 # km

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c

    return d


#print(distance(45.539284,-73.511116,45.539284,-73.511116))
#test entre 2 mêmes points, devrait être égal à 0

0.0


In [73]:
#contient la requête à l'adresse URL de la stm pour télecharger les données statiques
url_zip = urllib.request.urlopen("https://www.stm.info/sites/default/files/gtfs/gtfs_stm.zip")

#on lit le contenu du fichier zip
dossier_zippe = ZipFile(BytesIO(url_zip.read()))

#on ecrit sur le disque dur de l'ordinateur la base de données statique
#permet des calculs plus rapide que si on l'avait laissé dans la mémoire vive 
#plus simple pour utiliser les fichiers après 
dossier_zippe.extractall()

In [12]:
#URL qui nous permet de retirer les données en temps réel
#quant à la position en temps réel de tous les véhicules de la stm 
#ainsi que leur autres données 
url_vehicle = "https://api.stm.info/pub/od/gtfs-rt/ic/v1/vehiclePositions/"

#clé API qui nous offre un accès à la base de donnée, comme un mot de passe
headers = {'apikey': "l7xxb1d4a72c242f4709a4a96ca493b17e81"}

#l'utilisateur rentre le nom de la ligne de bus qui l'intéresse
#par exemple si il veut la ligne 51 en direction de l'Est, il rentre : 51-E
#ou si il veut la ligne 165 en direction du Nord, il rentre : 165-N
u_trip_headsign = input('Quel est votre ligne ? ')

#l'utilisateur rentre l'id de l'arret qui l'intéresse
u_stop_id = input("Quel est l'id de votre arret ? ")

#on convertie u_stop_id en entier pour les futurs opérations
u_stop_id = int(u_stop_id)



Quel est votre ligne ? 165-N
Quel est l'id de votre arret ? 51452


In [4]:
#on créee un dataframe à partir du fichier stop_times.txt avec comme séparateur une virgule
#on a ainsi pour chaque trip id, les stop_id auquels le bus doit aller 
#ainsi que leur ordre (stop_sequence)

df_stop_times = pandas.DataFrame(pandas.read_csv("stop_times.txt",sep=',',engine='python'))

In [5]:
#creation dataframe à partir de trips.txt avec comme séparateur une virgule
#colonnes intéressante:
#route_id service_id trip_id trip_headsign(ie:129-E) direction_id(0/1) shape_id

#etape la plus lourde/longue du programme peut prendre plusieurs secondes

df_trips = pandas.DataFrame(pandas.read_csv("trips.txt",sep=',',engine='python'))

In [10]:
#creation dataframe à partir de stops.txt avec comme séparateur une virgule
#contient les informations liées à un arret 
#colonnes intéressante:
#stop_id stop_name stop_lat stop_lon

df_stops = pandas.DataFrame(pandas.read_csv("stops.txt",sep=',',engine='python'))

In [20]:

feed_vehicle = gtfs_realtime_pb2.FeedMessage()

#on fait la requête pour avoir les données en temps réel, avec l'URL et la clé d'API initialisé avant
response_vehicle = requests.request("POST", url_vehicle, headers=headers)

#on analyse le flux de données et et on le retanscrit en quelquechose de lisible/sorte de json
feed_vehicle.ParseFromString(response_vehicle.content)

#on va chercher les latitude et longitude de notre arrêt rentré par l'utilisateur
#on va les ranger dans un numpy.ndarray(tableau) sorte de liste sans virgule
stop_coordonees = df_stops.loc[df_stops['stop_id']==str(u_stop_id)].values[0]
#on sort la latitude qui est la 4eme valeur
stop_lat = stop_coordonees[3]

#on sort la longitude qui est la 5eme valeur
stop_lon = stop_coordonees[4]

#si on veut voir les coordonnées pour différents test
#print('stop lat {0}'.format(stop_lat))
#print('stop lon {0}'.format(stop_lon))
#print(stop_coordonees)

In [19]:
#on va créer un dataframe à partir du flux de données en temps réel
#on va parcourir le flux avec une boucle for et créer à chaque itération un dataframe temporaire
#qu l'on va venir ajouter à un notre dataframe à completer

#on crée les bases du dataframe
#avec trois colonnes 'vehicle_id','trip_id' et 'current_stop_sequence'
df_temps_reel = pandas.DataFrame(columns=['vehicle_id','trip_id','current_stop_sequence'])

#on crée un itération à travers le flux de données en parcourant les entités parmis toutes 
#les entités dans le feed_vehicle.entity
for entity in feed_vehicle.entity:
    #on crée le dataframe temporaire en allouant à chaque colonnes les valeurs correspondantes
    #à partir du flux de données
    df_temps_reel_bis = pandas.DataFrame([(entity.id ,
                                           entity.vehicle.trip.trip_id ,
                                           entity.vehicle.current_stop_sequence,
                                           entity.vehicle.position.latitude,
                                           entity.vehicle.position.longitude)], 
                                         columns=['vehicle_id',
                                                  'trip_id',
                                                  'current_stop_sequence',
                                                  'current_latitude',
                                                  'current_longitude'])
    
    #on crée un liste qui contient les deux dataframe à fusionner
    frames = [df_temps_reel,df_temps_reel_bis]
    
    #on ajoute au bas du dataframe à compléter, le datafame provisoire 
    df_temps_reel = pandas.concat(frames, sort=True)
    
#on règle l'index sur trip_id pour nous faciliter la lecture et la vie plus tard pour les futurs
#fusions/filtre
df_temps_reel = df_temps_reel.set_index('trip_id')

#on trie les index dans l'ordre coissant
df_temps_reel = df_temps_reel.sort_index()

#test si on veut afficher les 5 premières lignes du dataframe
#df_temps_reel.head(5)

In [21]:
#vestige d'une ancienne façon de faire ou on utilisait tout le data frame, et surtout les trip_id
#mais maintenant on se contentera de récupere juste le stop sequence

#on crée un dataframe avec tous les trip_id et les stop_sequence 
#correspondant au stop_id de l'utilisateur 

df_stop_times_trié = (df_stop_times.loc[df_stop_times["stop_id"]==u_stop_id][['trip_id','stop_id','stop_sequence']])

#on veut que toutes les valeurs du dataframe soit des entiers
df_stop_times_trié = df_stop_times_trié.astype(int)

#on veut que l'index soit trip_id
df_stop_times_trié = df_stop_times_trié.set_index('trip_id')

#on trie par ordre croissant l'index
df_stop_times_trié = df_stop_times_trié.sort_index()

#on réinitialise l'index pour pouvoir récupérer les valeurs au rend 1 par exemple
df_stop_times_trié =df_stop_times_trié.reset_index()

#on crée un variable qui prend la valeur de la sequence de l'arret pour la ligne 
#rentré par l'utilisateur 
stop_sequence = df_stop_timess.at[0,'stop_sequence']

#pour tester on peut affcher les 5 premières lignes du dataframe
#df_stop_times_trié.head(5)

NameError: name 'df_stop_times' is not defined

In [17]:
#on crée un dataframe à partir du dataframe df_trips
#on veut tous les trip_id ayant pour trip_headsign celui rentré par l'utilisateur
df_trip_ids = (df_trips.loc[df_trips["trip_headsign"]==u_trip_headsign][['trip_id','trip_headsign']])

#on rêgle l'index sur trip_id
df_trip_ids = df_trip_ids.set_index('trip_id')

#on trie l'index
df_trip_ids = df_trip_ids.sort_index()

#pour tester on peut afficher les 5 premières lignes
#df_trip_ids.head(5)

,trip_headsign
trip_id,
202656775,51-E
202656778,51-E
202656781,51-E
202656838,51-E
202656862,51-E


In [18]:
#on merge un df_temps_reel et df_trip_ids 
#en fusionnaht, python ne retient que les id qui sont dans les 2 dataframe
#ainsi on a un nouveau data frame qui comporte les infos des bus en circulation
#en temps réel étant sur la ligne rentré par l'utilisateur
df_final1 = df_temps_reel.merge(df_trip_ids,left_index=True, right_index=True)

#pour tester on peut afficher le dataframe
#il est pas très grand, pas besoins d'utiliser head()
#df_final1

,current_latitude,current_longitude,current_stop_sequence,vehicle_id,trip_headsign
trip_id,,,,,
205186602,45.489216,-73.624214,26,36013,51-E
205186684,45.503117,-73.618919,33,36009,51-E
205186806,45.469131,-73.641953,12,30064,51-E
205186812,45.521282,-73.592499,48,37084,51-E
205186818,45.454235,-73.641487,1,36034,51-E


In [19]:
#on rajoute des informations nécessaires pour la suite, 
#la position de l'arret et sa séquence dans le trajet

#on crée des nouvelles colonnes que on va remplir avec la même valeur pou chaque ligne

df_final['stop_lon'] = stop_lon
df_final['stop_lat'] = stop_lat
df_final['stop_sequence'] = stop_sequence

#pour tester on peut afficher le data frame
#df_final

,current_latitude,current_longitude,current_stop_sequence,vehicle_id,trip_headsign,stop_lon,stop_lat,stop_sequence
trip_id,,,,,,,,
205186602,45.489216,-73.624214,26,36013,51-E,-73.589934,45.523775,49
205186684,45.503117,-73.618919,33,36009,51-E,-73.589934,45.523775,49
205186806,45.469131,-73.641953,12,30064,51-E,-73.589934,45.523775,49
205186812,45.521282,-73.592499,48,37084,51-E,-73.589934,45.523775,49
205186818,45.454235,-73.641487,1,36034,51-E,-73.589934,45.523775,49


In [20]:
#on crée un nouvelle colonne calculé pour savoir le nombre d'arret qu'il reste au bus à parcourir 
#jusque à l'arret de l'utilisateur
#on soustrait alors la sequence de l'arret avec la position du bus dans la sequence
df_final['distance_arret'] = df_final['stop_sequence']-df_final['current_stop_sequence']

#on affiche que quand distance_arret est plus grand que 0
#c'est à dire quand le bus n'est pas encore passé
#quand = 0, le bus est au même arret que l'utilisateur
#quand<0, le bus est déjà passé et il ne nous interesse plus
df_final = df_final.loc[df_final['distance_arret']>0]

#pour tester on peut afficher le dataframe
#df_final

#le code suivant, c'est un peu du bricollage, puisque:
#df_final['distance_km']=distance(df_final['stop_lat'],df_final['stop_lon'],
                                    #df_final['current_latitude'],df_final['current_longitude'])
#ne fonctionne pas, on a du improviser n'arivant pas à surpasser le problème

#on parcour les lignes du dataframe et on recupere l'index et la ligne
for index, row in df_final.iterrows():
    #on assigne à une variable la valeur du calcul de la distance en utilisant la fonction 
    #distance, definie au début du code
    dist =distance(row['stop_lat'],row['stop_lon'],row['current_latitude'],row['current_longitude'])
    #on arrondi la distance
    dist = round(dist, 2)

    #on assigne dans le dataframe dans la colonne et la ligne correespondante la valeur de dist 
    df_final.at[index,'distance_km'] = dist

#on peut tester en affichant le dataframe après les modifications    
#df_final


#on trie en ordre croissant selon la séquence pour avoir les plus proches le plus haut
df_final = df_final.sort_values('distance_arret')

#on reinitialise l'index pour acceder facilment aux données avec des chiffres 0,1,2, sans avoir
#à connaitre les trip_id
df_final = df_final.reset_index()

#on affiche le dataframe final pour vérifier le tout 
df_final



,trip_id,current_latitude,current_longitude,current_stop_sequence,vehicle_id,trip_headsign,stop_lon,stop_lat,stop_sequence,calcule,distance
0,205186812,45.521282,-73.592499,48,37084,51-E,-73.589934,45.523775,49,1,0.341698
1,205186684,45.503117,-73.618919,33,36009,51-E,-73.589934,45.523775,49,16,3.221424
2,205186602,45.489216,-73.624214,26,36013,51-E,-73.589934,45.523775,49,23,4.680121
3,205186806,45.469131,-73.641953,12,30064,51-E,-73.589934,45.523775,49,37,7.304612
4,205186818,45.454235,-73.641487,1,36034,51-E,-73.589934,45.523775,49,48,8.714434


In [21]:
#les variables destinées à folium commencent par un f

#coordonnées pour folium de l'arret de bus
#on accède à la valeur du data frame à la première ligne pour la colonne stop_lat
f_stop_lat = df_final.at[0,'stop_lat']
#on accède à la valeur du data frame à la première ligne pour la colonne stop_lon
f_stop_lon = df_final.at[0,'stop_lon']

#coordonnées pour folium du bus le plus proche de l'arrêt de bus
#on accède à la valeur du data frame à la première ligne pour la colonne current_latitude
f_current_lat = df_final.at[0,'current_latitude']

#on accède à la valeur du data frame à la première ligne pour la colonne current_longitude
f_current_lon = df_final.at[0,'current_longitude']

#on recupère la valeur de la distance entre le bus le plus proche et l'arret de bus 
f_distance = df_final3.at[0, 'distance']

#on crée la carte centrée autour des coordonnées de l'arret de bus avec un zoom pas trop gros mais 
#pas trop faible non plus pou qu'on puisse voir le bus même s il n'est pas à coté
m = folium.Map(location=[f_stop_lat, f_stop_lon], zoom_start=15)

#on crée un marqueur pour le bus
#on lui ajoute une étiquette pour ajouter une estimation de la distance
folium.Marker(
    location=[f_current_lat, f_current_lon],
    popup='BUS distance: ' + '{0}'.format (f_distance) + ' km',
    icon=folium.Icon(icon='flower')
).add_to(m)


#on crée un marqueur pour l'arret de bus, il devrait être centré sur la carte 
folium.Marker(
    location=[f_stop_lat, f_stop_lon],
    popup='Timberline Lodge',
    icon=folium.Icon(color='green')
).add_to(m)

m

In [150]:
#on crée un faux data frame pour faire des test
#d = {'trip_id' : ['205018150','205017943'],'stop_id' : [u_stop_id,u_stop_id],'stop_sequence' : [50,4]}
#df_fake = pandas.DataFrame(data = d)
#df_fake = df_fake.set_index('trip_id')
#df_fake

In [134]:
for entity in feed_vehicle.entity:
    if entity.vehicle.trip.route_id == "144":
        print(entity.vehicle)

trip {
  trip_id: "205187163"
  start_time: "17:34:00"
  start_date: "20191201"
  route_id: "144"
}
position {
  latitude: 45.51539993286133
  longitude: -73.57493591308594
}
current_stop_sequence: 4
current_status: IN_TRANSIT_TO
timestamp: 1575239914
vehicle {
  id: "36033"
}

trip {
  trip_id: "205187095"
  start_time: "17:35:00"
  start_date: "20191201"
  route_id: "144"
}
position {
  latitude: 45.4951171875
  longitude: -73.58858489990234
}
current_stop_sequence: 6
current_status: IN_TRANSIT_TO
timestamp: 1575239913
vehicle {
  id: "36040"
}



In [135]:
print(feed_vehicle.entity)

[id: "22282"
is_deleted: false
vehicle {
  trip {
    trip_id: "204973242"
    start_time: "17:10:00"
    start_date: "20191201"
    route_id: "496"
  }
  position {
    latitude: 45.43848419189453
    longitude: -73.66621398925781
  }
  current_stop_sequence: 20
  current_status: IN_TRANSIT_TO
  timestamp: 1575239904
  vehicle {
    id: "22282"
  }
}
, id: "22324"
is_deleted: false
vehicle {
  trip {
    trip_id: "204971340"
    start_time: "17:32:00"
    start_date: "20191201"
    route_id: "113"
  }
  position {
    latitude: 45.44580078125
    longitude: -73.59648132324219
  }
  current_stop_sequence: 3
  current_status: STOPPED_AT
  timestamp: 1575239920
  vehicle {
    id: "22324"
  }
}
, id: "22328"
is_deleted: false
vehicle {
  trip {
    trip_id: "204971414"
    start_time: "17:28:00"
    start_date: "20191201"
    route_id: "106"
  }
  position {
    latitude: 45.43265151977539
    longitude: -73.62986755371094
  }
  current_stop_sequence: 12
  current_status: STOPPED_AT
  ti